In [1]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from collections import deque 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import plotly.express as px

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
X,y = make_classification(15_000,500,random_state=50,weights=[0.9])
X, X_validate, y, y_validate = train_test_split(X,y, random_state=50)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =  0.7)

In [292]:
X_train_ = X_train.copy()
y_train_ = y_train.copy()
rng_idx = pd.Series(y_train).sample(len(y_train)).index
X_train_ = X_train_[rng_idx]
y_train_ = y_train_[rng_idx]
M_idx, m_idx = pd.DataFrame(y_train_).groupby(0).apply(lambda x: x.index.values)

In [230]:
# pd.Series(y_train).value_counts()   

In [231]:
# bal_idx = pd.DataFrame(y_train, columns = ['y']).groupby('y').apply(lambda x: x.sample(250)).index.to_frame()[1].values


In [232]:
# rng = np.random.default_rng(1235)

In [233]:
# rng.shuffle(bal_idx)

In [234]:
# X_bal = X_train[bal_idx]
# y_bal = y_train[bal_idx]

In [235]:
# X_bal = X_train.copy()
# y_bal = y_train.copy()

In [236]:
# M_idx, m_idx = pd.DataFrame(y_bal).groupby(0).apply(lambda x: x.index.values)

---

In [8]:
 actions = {0 : 0, 1 : 1}
 len(actions)

2

In [262]:
%run -i ./DQN_Class.py
gamma=0.5
epsilon=.8
numberEpisodes= 120
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


# NN approach

In [263]:

nn = LearningQDeep.buildNetwork()

In [264]:
nn.summary()

Model: "sequential_95"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_355 (Dense)           (None, 30)                15030     
                                                                 
 dense_356 (Dense)           (None, 64)                1984      
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_357 (Dense)           (None, 20)                1300      
                                                                 
 dense_358 (Dense)           (None, 1)                 21        
                                                                 
Total params: 18,335
Trainable params: 18,335
Non-trainable params: 0
_________________________________________________________________


In [265]:

early_stopping = tf.keras.callbacks.EarlyStopping(
                                monitor='val_f1', 
                                verbose=1,
                                patience=100,
                                mode='max',
                                restore_best_weights=True)

In [266]:
nn.fit(X_train,y_train, validation_data=[X_test,y_test], epochs=500, callbacks = [early_stopping])

Epoch 1/500
106/106 [==============================] - 2s 7ms/step - loss: 0.4291 - recall_m: 0.1144 - precision_m: 0.0462 - f1: 0.0513 - val_loss: 0.3416 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/500
106/106 [==============================] - 0s 4ms/step - loss: 0.3112 - recall_m: 0.0267 - precision_m: 0.0943 - f1: 0.0398 - val_loss: 0.3233 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 3/500
106/106 [==============================] - 0s 5ms/step - loss: 0.2560 - recall_m: 0.0598 - precision_m: 0.1462 - f1: 0.0794 - val_loss: 0.3043 - val_recall_m: 0.0095 - val_precision_m: 0.0283 - val_f1: 0.0128
Epoch 4/500
106/106 [==============================] - 0s 4ms/step - loss: 0.2061 - recall_m: 0.2113 - precision_m: 0.4371 - f1: 0.2641 - val_loss: 0.2875 - val_recall_m: 0.1013 - val_precision_m: 0.2618 - val_f1: 0.1369
Epoch 5/500
106/106 [==============================] - 0s 5ms/step - loss: 0.1515 - recall_m

In [272]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_train)]

106/106 [==============================] - 0s 2ms/step


In [273]:
print(confusion_matrix(y_train, y_pred))

[[3056    0]
 [   0  319]]


In [274]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3056
           1       1.00      1.00      1.00       319

    accuracy                           1.00      3375
   macro avg       1.00      1.00      1.00      3375
weighted avg       1.00      1.00      1.00      3375



In [275]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_test)]

247/247 [==============================] - 0s 2ms/step


In [276]:
print(confusion_matrix(y_test, y_pred))

[[6795  225]
 [ 438  417]]


In [277]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7020
           1       0.65      0.49      0.56       855

    accuracy                           0.92      7875
   macro avg       0.79      0.73      0.76      7875
weighted avg       0.91      0.92      0.91      7875



In [278]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_validate)]

118/118 [==============================] - 0s 1ms/step


In [279]:
print(confusion_matrix(y_validate, y_pred))

[[3255  118]
 [ 191  186]]


In [280]:
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      3373
           1       0.61      0.49      0.55       377

    accuracy                           0.92      3750
   macro avg       0.78      0.73      0.75      3750
weighted avg       0.91      0.92      0.91      3750



---

In [14]:
def reward(si, a, M, m, y):
    l = y[si]

    if (a == l) & (si in m):
        r = 1
    elif  (a != l) & (si in m):
        r = -1
    if (a == l) & (si in M):
        r = 0.5
    elif  (a != l) & (si in M):
        r = -0.5
    return r

In [15]:
def step(action, i, M, m, X, y):

    r = reward(i,action, M, m, y)

    nextState = X[i+1,:]

    terminated = False
    
    l = y[i]

    if i == len(y) - 1:
        terminated = True
    elif (action != l) & (i in m):
        print(action, l)
        terminated = True


    return r, nextState, terminated, i+1
    

In [20]:
%run -i ./DQN_Class.py
gamma=0.2
epsilon=.9
numberEpisodes= 500
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


In [21]:
0.96**(500-25)*(0.9)

3.4125435562551007e-09

In [22]:
LearningQDeep.actionDimension

2

In [23]:
LearningQDeep.epsilon

0.9

In [24]:
LearningQDeep.mainNetwork.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 30)                15030     
                                                                 
 dense_20 (Dense)            (None, 64)                1984      
                                                                 
 dense_21 (Dense)            (None, 20)                1300      
                                                                 
 dense_22 (Dense)            (None, 2)                 42        
                                                                 
Total params: 18,356
Trainable params: 18,356
Non-trainable params: 0
_________________________________________________________________


In [25]:
%%time
LearningQDeep.trainingEpisodes()

Simulating episode 0
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 2 21 47 51 58 69 70 74 78 81]
	 Reached sample: 22
	 Sum of rewards 0.0
Simulating episode 1
	 Fist train of main network...
0 1
	 First 1 index: [  5   9  25  58  61  68  69  98 113 117]
	 Reached sample: 10
	 Sum of rewards 0.0
Simulating episode 2
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 19  43  49  54  59  70  72  84  89 101]
	 Reached sample: 20
	 Sum of rewards -0.5
Simulating episode 3
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
		 [UPDATED] Target network
		 Current epsilon: 0.9
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 16  26  34  49  57 124 132 143 162 177]
	 Reached sample: 58
	 Sum of rewards 1.5
Simulating episode 4
	 Fist train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 

In [26]:
loaded_model = LearningQDeep.mainNetwork

In [27]:
y_pred  = np.argmax(loaded_model.predict(X_train,verbose=0), axis = 1)

In [28]:
print(confusion_matrix(y_train, y_pred))

[[2988   68]
 [  56  263]]


In [29]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3056
           1       0.79      0.82      0.81       319

    accuracy                           0.96      3375
   macro avg       0.89      0.90      0.89      3375
weighted avg       0.96      0.96      0.96      3375



In [30]:
y_pred  = np.argmax(loaded_model.predict(X_test,verbose=0), axis = 1)

In [31]:
print(confusion_matrix(y_test, y_pred))

[[6761  259]
 [ 438  417]]


In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      7020
           1       0.62      0.49      0.54       855

    accuracy                           0.91      7875
   macro avg       0.78      0.73      0.75      7875
weighted avg       0.90      0.91      0.91      7875



In [33]:
y_pred  = np.argmax(loaded_model.predict(X_validate,verbose=0), axis = 1)

In [34]:
print(confusion_matrix(y_validate, y_pred))

[[3250  123]
 [ 194  183]]


In [35]:
print(classification_report(y_validate,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3373
           1       0.60      0.49      0.54       377

    accuracy                           0.92      3750
   macro avg       0.77      0.72      0.74      3750
weighted avg       0.91      0.92      0.91      3750



In [36]:
LearningQDeep.mainNetwork.save('Trained_model.h5')